In [ ]:
import numpy as np
import pandas as pd

In [ ]:
bus_groups_time_series = pd.read_csv("../data/bus_groups_timeseries.csv", index_col=0)
bus_groups_time_series

In [ ]:
# ①5000kmごとにbinの番号をつける。
def calculate_bin_or_keep_nan(val):
    """
    valがNaNであれば、NaNを返します。
    そうでなければ、valを5000kmごとのビンに分類し、そのビンの番号を返します。
    ただし、ビンの番号が21以上の場合は21を返します。
    """
    if pd.isna(val):
        return np.nan
    else:
        bin_number = np.floor(val / 5000).astype(int) + 1
        return min(bin_number, 21)


numbers = bus_groups_time_series.map(calculate_bin_or_keep_nan)
numbers

In [ ]:
# ②a_iを求める（＝diffを取る）
diffs = numbers.diff(axis=0)
diffs

In [ ]:
# ③x1-x4を求める：最後の行がx4なので特別処理。残りはcountすれば良い。
last_row = diffs.iloc[-1]
diffs = diffs.iloc[:-1]

value_counts_per_column = diffs.apply(pd.Series.value_counts, axis=0)
value_counts_per_column.loc["last_row"] = last_row
value_counts_per_column = value_counts_per_column.rename(
    index={0: "x1", 1: "x2", 2: "x3", "last_row": "x4"}
)
value_counts_per_column.fillna(0, inplace=True)
sums = value_counts_per_column.sum(axis=1)
sums

In [ ]:
# ④kappa1とkappa2を計算する。dfの形で出力する。
def kappa1(x1, x2, x3, x4):
    return x2 * (x2 + x3 + x4) / (x2 + x3) * (x1 + x2 + x3 + x4)


def kappa2(x1, x2, x3, x4):
    return x3 * (x2 + x3 + x4) / (x2 + x3) * (x1 + x2 + x3 + x4)


kappa1 = kappa1(sums["x1"], sums["x2"], sums["x3"], sums["x4"])
kappa2 = kappa2(sums["x1"], sums["x2"], sums["x3"], sums["x4"])

kappa1, kappa2